<a href="https://colab.research.google.com/github/yoichinaka/Pneumonia-detector-by-Xray-image/blob/master/Pneumonia_detector_by_Xray_image_using_transfer_learning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [2]:
#!pip install kaggle

In [1]:
from google.colab import files
files.upload()

{}

In [0]:
!mkdir -p ~/.kaggle  #kaggleのファイルを作る
!cp kaggle.json ~/.kaggle  #kaggleのトークンをコピーする

#kaggleのアクセス権を変更する
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!ls

a.jpeg			  chest_xray.zip  __MACOSX
chest_xray		  drive		  model_after_10_epochs
chest-xray-pneumonia.zip  kaggle.json	  sample_data


In [0]:
#!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 98% 1.13G/1.15G [00:12<00:00, 80.2MB/s]
100% 1.15G/1.15G [00:12<00:00, 97.9MB/s]


In [0]:
from zipfile import ZipFile
file_name = 'chest_xray.zip'

with ZipFile(file_name , 'r') as zip_file:
  zip_file.extractall()
  print('done')

done


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os, math, json
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
tf.enable_eager_execution()
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 1.14.0


In [0]:
from IPython.display import clear_output

%matplotlib inline

In [0]:
from pathlib import Path

#train_files = "chest_xary/train/"
train_files = "drive/My Drive/train"
#test_files = "chest_xar/test/"
test_files = "drive/My Drive/test"

positive_class_folder_name = "PNEUMONIA"
negative_class_folder_name = "NORMAL"

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(featurewise_center = True,
                                     shear_range = 0.2,
                                     rescale = 1.0/255,
                                     horizontal_flip = True,
                                     validation_split = 0.2)

test_generator = ImageDataGenerator(rescale = 1.0/255)

In [16]:
train_iterator = train_generator.flow_from_directory(train_files, 
                                                     target_size = (150, 150),
                                                     classes = [positive_class_folder_name, negative_class_folder_name],
                                                     batch_size = 256, 
                                                     shuffle = True, 
                                                     class_mode = "binary",
                                                     subset = "training")
                                                    
validation_iterator = train_generator.flow_from_directory(train_files, 
                                                          target_size = (150, 150),
                                                          classes = [positive_class_folder_name, negative_class_folder_name],
                                                          batch_size = 128, 
                                                          shuffle = False, 
                                                          class_mode = "binary",
                                                          subset = "validation")

test_iterator = test_generator.flow_from_directory(test_files, 
                                                   target_size = (150, 150), 
                                                   classes = [positive_class_folder_name, negative_class_folder_name],
                                                   batch_size = 128, 
                                                   shuffle = False,
                                                   class_mode = "binary")

Found 4194 images belonging to 2 classes.
Found 1047 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
# count for class instances in training data

class1 = sum(train_iterator.classes)
class0 = len(train_iterator.classes) - class1

print("Label 0 - %i, Label 1 - %i" % (class0, class1))

Label 0 - 3106, Label 1 - 1088


In [7]:
len(train_iterator.classes)

4194

In [10]:
from sklearn.utils.class_weight import compute_class_weight

y = train_iterator.classes
labels = np.unique(y)

train_class_weights = compute_class_weight("balanced", labels, y)
train_class_weights

array([0.67514488, 1.92738971])

In [0]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, BatchNormalization, Dense, Flatten
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [12]:
IMAGE_SIZE = [150, 150]
#pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False)
pretrained_model = InceptionV3(weights = "imagenet", include_top = False, input_shape = [*IMAGE_SIZE, 3])
#pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
#pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
#pretrained_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
pretrained_model.trainable = False

### QUESTION 1.
model = tf.keras.Sequential([
    pretrained_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation = "relu"),
    Dropout(0.3),
    Dense(512, activation = "relu"),
    Dropout(0.3),
    Dense(256, activation = "relu"),
    Dropout(0.3),
    Dense(128, activation = "relu"),
    Dropout(0.3),
    Dense(1, activation = "sigmoid")])

model.compile(
    optimizer='adam',
    loss = "binary_crossentropy",
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [22]:
clear_session()

base_model = InceptionV3(weights = "imagenet", include_top = False, input_shape = (150, 150, 3))
clear_output()

base_model.trainable = False
myModel = Sequential([base_model,
                      GlobalAveragePooling2D(),
                      Dense(1024, activation = "relu"),
                      Dropout(0.3),
                      Dense(512, activation = "relu"),
                      Dropout(0.3),
                      Dense(256, activation = "relu"),
                      Dropout(0.3),
                      Dense(128, activation = "relu"),
                      Dropout(0.3),
                      Dense(1, activation = "sigmoid")])

myModel.compile(RMSprop(), loss = "binary_crossentropy", metrics = ["accuracy"])
myModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [35]:
!rmdir /drive/My Drive/train/NORMAL1

rmdir: failed to remove '/drive/My': No such file or directory
rmdir: failed to remove 'Drive/train/NORMAL1': No such file or directory


In [17]:
history = model.fit_generator(train_iterator, 
                                steps_per_epoch = len(train_iterator),
                                epochs = 10,
                                verbose = 1,
                                workers = 20,
                                validation_data = validation_iterator, 
                                validation_steps = len(validation_iterator),
                                class_weight = train_class_weights)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


17/17 [==============================] - 117s 7s/step - loss: 0.6315 - acc: 0.6941 - val_loss: 0.5134 - val_acc: 0.7689
Epoch 2/10
17/17 [==============================] - 58s 3s/step - loss: 0.3650 - acc: 0.8429 - val_loss: 0.4172 - val_acc: 0.8758
Epoch 3/10
17/17 [==============================] - 63s 4s/step - loss: 0.2520 - acc: 0.8996 - val_loss: 0.3441 - val_acc: 0.8768
Epoch 4/10
17/17 [==============================] - 66s 4s/step - loss: 0.2037 - acc: 0.9225 - val_loss: 0.4436 - val_acc: 0.8271
Epoch 5/10
17/17 [==============================] - 64s 4s/step - loss: 0.1902 - acc: 0.9294 - val_loss: 0.4159 - val_acc: 0.8577
Epoch 6/10
17/17 [==============================] - 63s 4s/step - loss: 0.1767 - acc: 0.9320 - val_loss: 0.4500 - val_acc: 0.8481
Epoch 7/10
17/17 [==============================] - 64s 4s/step - loss: 0.1645 - acc: 0.9354 - val_loss: 0.3768 - val_acc: 0.8816
Epoch 8/10
17/17 [==============================] - 66s 4s/step - loss: 0.1742 - acc: 0.9280 - val_l

In [0]:
save_model(model, "./model_after_10_epochs", overwrite = True, include_optimizer = True)

In [0]:
#model =load_model("./model_after_10_epochs")

In [20]:
!ls

a.jpeg			  chest_xray.zip  __MACOSX
chest_xray		  drive		  model_after_10_epochs
chest-xray-pneumonia.zip  kaggle.json	  sample_data


In [0]:
yhat = model.predict_generator(test_iterator,
                                 steps = len(test_iterator),
                                 verbose = 0)  

In [0]:
mask = yhat > 0.5
yhat[mask] = 1
yhat[~mask] = 0
y = test_iterator.classes

In [23]:
from sklearn.metrics import classification_report

target_names = ['Pneumonia', 'Normal']
print(classification_report(y, yhat, target_names = target_names))

              precision    recall  f1-score   support

   Pneumonia       0.71      0.92      0.80       390
      Normal       0.72      0.36      0.48       234

    accuracy                           0.71       624
   macro avg       0.71      0.64      0.64       624
weighted avg       0.71      0.71      0.68       624

